# Horticultural Data Integration Notebook

This notebook is designed to integrate scientific plant names from a local database with information from the World Checklist of Vascular Plants (WCVP) and the USDA. The goal is to enrich the local plant data with taxonomic and other relevant information from these external sources.

## Notebook Steps:

1.  **Setup**: Imports necessary libraries and defines file paths.
2.  **File Specification**: Specifies the input and output filenames for the datasets.
3.  **WCVP Taxon Status Filter**: Defines the preferred order of taxon statuses from the WCVP dataset for deduplication.
4.  **Load and Parse Scientific Names**: Reads the local scientific names, extracts unique names, and parses them into taxonomic components (genus, species, etc.).
5.  **Load and Filter WCVP Names**: Reads the WCVP dataset, filters based on the defined taxon statuses, and adds a prefix to column names.
6.  **Load and Parse USDA Names**: Reads the USDA dataset, removes unnecessary columns, parses scientific names, and adds a prefix to column names.
7.  **Join Datasets**: Merges the three datasets (local, WCVP, and USDA) based on the parsed scientific name components.
8.  **Deduplicate WCVP Entries**: Orders the merged data by scientific name and WCVP taxon status to keep the preferred entry for each scientific name.
9.  **Select Final Columns**: Selects the desired columns for the final output dataframe and renames the USDA common name column.
10. **Clean WCVP Published Date**: Removes parentheses from the WCVP first published date column.
11. **Display and Save Final Data**: Displays the head of the final dataframe and saves it to a CSV file.

In [24]:
import pandas as pd

project_path = '/content/drive/project/'
project_data_path = project_path + 'data/'

import os
os.chdir(project_path)

#module to parse scientifc plant names
import sci_name_parser as snap

In [25]:
#Specify the source dataset file names

#input scientific names
DB_scientific_names_file = 'plant_scientific_names_test_data.csv'

#World Checklist of Vascular Plants information
WCVP_file = 'wcvp_names.csv'

#U.S. Dept. of Agriculture plant information
USDA_file = 'usda_plants.csv'


In [26]:
#Select the taxon status to filter for, and the order to deduplicate, with the left-most being retained in the final dataframe.
#taxon_status: 'Synonym', 'Accepted', 'Unplaced', 'Artificial Hybrid', 'Illegitimate', 'Invalid', 'Orthographic', 'Local Biotype', 'Misapplied'
#An empty string '' should be the last item on the list.

WCVP_taxon_status_filter = ['Accepted','Artificial Hybrid','Synonym','']

In [27]:
#read the input scientific names and parse them

df_scientific_names = pd.read_csv(project_data_path + DB_scientific_names_file, keep_default_na=False, dtype=object)

df_scientific_names_unique = df_scientific_names['scientific_name'].str.strip().drop_duplicates().to_frame()

df_scientific_names_unique = df_scientific_names_unique.apply(snap.generate_scientific_columns, axis=1, scientific_name_col='scientific_name').add_prefix('DB_')

display(df_scientific_names_unique.head())


,DB_scientific_name,DB_taxon_rank,DB_genus,DB_species,DB_genus_hybrid,DB_species_hybrid,DB_infraspecies,DB_variety,DB_subspecies,DB_form
0,Heptacodium miconioides,Species,Heptacodium,miconioides,,,,,,
1,Trillium grandiflorum f. roseum,Form,Trillium,grandiflorum,,,roseum,,,roseum
2,Chrysogonum virginianum var. australe,Variety,Chrysogonum,virginianum,,,australe,australe,,
3,×Solidaster luteus,Species,Solidaster,luteus,×,,,,,
4,Platanus × hispanica,Species,Platanus,hispanica,,×,,,,


In [29]:
#Read the WCVP names and filter them for particular taxon status

df_WCVP_names=pd.read_csv(project_data_path + WCVP_file, sep='|', keep_default_na=False, dtype=object).add_prefix('WCVP_')

#apply the taxon status filter
df_WCVP_names_taxon_stat = df_WCVP_names[df_WCVP_names['WCVP_taxon_status'].isin(WCVP_taxon_status_filter)]

display(df_WCVP_names_taxon_stat.head())

,WCVP_plant_name_id,WCVP_ipni_id,WCVP_taxon_rank,WCVP_taxon_status,WCVP_family,WCVP_genus_hybrid,WCVP_genus,WCVP_species_hybrid,WCVP_species,WCVP_infraspecific_rank,WCVP_infraspecies,WCVP_parenthetical_author,WCVP_primary_author,WCVP_publication_author,WCVP_place_of_publication,WCVP_volume_and_page,WCVP_first_published,WCVP_nomenclatural_remarks,WCVP_geographic_area,WCVP_lifeform_description,WCVP_climate_description,WCVP_taxon_name,WCVP_taxon_authors,WCVP_accepted_plant_name_id,WCVP_basionym_plant_name_id,WCVP_replaced_synonym_author,WCVP_homotypic_synonym,WCVP_parent_plant_name_id,WCVP_powo_id,WCVP_hybrid_formula,WCVP_reviewed
0,3018447,77112035-1,Species,Synonym,Melastomataceae,,Behuria,,magdalenensis,,,Brade,R.Tav. & Baumgratz,,Rodriguésia,61: 149,(2010),,,,,Behuria magdalenensis,(Brade) R.Tav. & Baumgratz,3236084,3024122,,T,,77112035-1,,N
1,3011086,60473329-2,Species,Accepted,Pittosporaceae,,Rhytidosporum,,procumbens,,,Hook.,F.Muell.,,Pl. Victoria,1: 75,(1862),,E. & SE. Australia,subshrub,subtropical,Rhytidosporum procumbens,(Hook.) F.Muell.,3011086,2571583,,,3011085,60473329-2,,N
2,108386,972411-1,Species,Accepted,Rubiaceae,,Larsenaikia,,suffruticosa,,,R.Br. ex Benth.,Govaerts,,Skvortsovia,4(3): 83,(2018),,N. Northern Territory,shrub,seasonally dry tropical,Larsenaikia suffruticosa,(R.Br. ex Benth.) Govaerts,108386,88479,,,108383,972411-1,,Y
3,3052127,1053324-2,Species,Accepted,Asteraceae,,Vernonanthura,,cordata,,,Kunth,H.Rob.,,Phytologia,73: 70,(1992),,C. & SW. Mexico,,,Vernonanthura cordata,(Kunth) H.Rob.,3052127,3052123,,,3134351,1053324-2,,N
4,3302520,77333177-1,Species,Accepted,Apocynaceae,,Chthamalia,,texensis,,,Correll,Morillo,,Mem. Fund. La Salle Ci. Nat.,81(191): 56,(2023),,SW. Texas,climber,desert or dry shrubland,Chthamalia texensis,(Correll) Morillo,3302520,508734,,,503106,77333177-1,,Y


In [31]:
%%capture
#read and parse the USDA names

df_USDA_names = pd.read_csv(project_data_path + USDA_file, keep_default_na=False, dtype=object)

#eliminate unneeded columns
df_USDA_names = df_USDA_names.drop(columns=['Symbol', 'Synonym Symbol', 'Family'])

df_USDA_names = df_USDA_names.apply(snap.generate_scientific_columns, axis=1, scientific_name_col='Scientific Name with Author', ignore_form=True).add_prefix('USDA_')

display(df_USDA_names.head())

In [33]:
#join the 3 datasets using the parsed scientific name columns

df_DB_WCVP_USDA = df_scientific_names_unique.merge(df_WCVP_names_taxon_stat, how='left',
               left_on=
               ['DB_taxon_rank',
               'DB_genus',
               'DB_species',
               'DB_genus_hybrid',
               'DB_species_hybrid',
               'DB_infraspecies'
               ],
               right_on=
               ['WCVP_taxon_rank',
               'WCVP_genus',
               'WCVP_species',
               'WCVP_genus_hybrid',
               'WCVP_species_hybrid',
               'WCVP_infraspecies'
               ]).fillna('').merge(df_USDA_names, how='left',
               left_on=
               ['DB_taxon_rank',
               'DB_genus',
               'DB_species',
               'DB_genus_hybrid',
               'DB_species_hybrid',
               'DB_infraspecies'
               ],
               right_on=
               ['USDA_taxon_rank',
               'USDA_genus',
               'USDA_species',
               'USDA_genus_hybrid',
               'USDA_species_hybrid',
               'USDA_infraspecies'
                ]).fillna('')

In [34]:
#A plant can have multiple entries in the WCVP, for example as both Accepted and Synonym taxon status.
#Filter the joined data set to drop duplicates based on scientific name and taxon status order.

df_DB_WCVP_USDA['WCVP_taxon_status'] = pd.Categorical(df_DB_WCVP_USDA['WCVP_taxon_status'], categories=WCVP_taxon_status_filter, ordered=True)

df_DB_WCVP_USDA_sorted = df_DB_WCVP_USDA.sort_values(['DB_scientific_name','WCVP_taxon_status'])

df_DB_WCVP_USDA_sorted_deduped = df_DB_WCVP_USDA_sorted.drop_duplicates(subset=['DB_scientific_name'], keep='first')

In [35]:
#select the columns for the final dataframe

WCVP_columns_final = ['DB_scientific_name',
                      'USDA_Common Name',
                      'WCVP_plant_name_id',
                      'WCVP_ipni_id',
                      'WCVP_taxon_rank',
                      'WCVP_taxon_status',
                      'WCVP_family',
                      'WCVP_genus_hybrid',
                      'WCVP_genus',
                      'WCVP_species_hybrid',
                      'WCVP_species',
                      'WCVP_infraspecific_rank',
                      'WCVP_infraspecies',
                      'WCVP_parenthetical_author',
                      'WCVP_primary_author',
                      'WCVP_publication_author',
                      'WCVP_place_of_publication',
                      'WCVP_volume_and_page',
                      'WCVP_first_published',
                      'WCVP_nomenclatural_remarks',
                      'WCVP_geographic_area',
                      'WCVP_lifeform_description',
                      'WCVP_climate_description',
                      'WCVP_taxon_name',
                      'WCVP_taxon_authors',
                      'WCVP_accepted_plant_name_id',
                      'WCVP_basionym_plant_name_id',
                      'WCVP_replaced_synonym_author',
                      'WCVP_homotypic_synonym',
                      'WCVP_parent_plant_name_id',
                      'WCVP_powo_id',
                      'WCVP_hybrid_formula',
                      'WCVP_reviewed']

df_DB_WCVP_USDA_final = df_DB_WCVP_USDA_sorted_deduped[WCVP_columns_final].rename(columns={'USDA_Common Name': 'USDA_common_name'}).reset_index(drop=True)

In [36]:
#remove parentheses from the date values
df_DB_WCVP_USDA_final['WCVP_first_published'] = df_DB_WCVP_USDA_final['WCVP_first_published'].str.replace('(','').str.replace(')','')

In [37]:
display(df_DB_WCVP_USDA_final.head())

,DB_scientific_name,USDA_common_name,WCVP_plant_name_id,WCVP_ipni_id,WCVP_taxon_rank,WCVP_taxon_status,WCVP_family,WCVP_genus_hybrid,WCVP_genus,WCVP_species_hybrid,WCVP_species,WCVP_infraspecific_rank,WCVP_infraspecies,WCVP_parenthetical_author,WCVP_primary_author,WCVP_publication_author,WCVP_place_of_publication,WCVP_volume_and_page,WCVP_first_published,WCVP_nomenclatural_remarks,WCVP_geographic_area,WCVP_lifeform_description,WCVP_climate_description,WCVP_taxon_name,WCVP_taxon_authors,WCVP_accepted_plant_name_id,WCVP_basionym_plant_name_id,WCVP_replaced_synonym_author,WCVP_homotypic_synonym,WCVP_parent_plant_name_id,WCVP_powo_id,WCVP_hybrid_formula,WCVP_reviewed
0,Acalypha virginica,Virginia threeseed mercury,1359,1586-2,Species,Accepted,Euphorbiaceae,,Acalypha,,virginica,,,,L.,,Sp. Pl.,: 1003,1753,", nom. cons.",C. & E. U.S.A.,annual,temperate,Acalypha virginica,L.,1359,,,,126,1586-2,,Y
1,Acer pensylvanicum,striped maple,2616210,781446-1,Species,Accepted,Sapindaceae,,Acer,,pensylvanicum,,,,L.,,Sp. Pl.,: 1055,1753,,SE. Canada to NC. & E. U.S.A.,tree,temperate,Acer pensylvanicum,L.,2616210,,,,2615354,781446-1,,N
2,Acer rubrum,red maple,2616329,1867-2,Species,Accepted,Sapindaceae,,Acer,,rubrum,,,,L.,,Sp. Pl.,: 1055,1753,,E. Canada to C. & E. U.S.A.,tree,temperate,Acer rubrum,L.,2616329,,,,2615354,1867-2,,N
3,Acer saccharinum,silver maple,2616367,1892-2,Species,Accepted,Sapindaceae,,Acer,,saccharinum,,,,L.,,Sp. Pl.,: 1055,1753,,SE. Canada to NC. & E. U.S.A.,tree,temperate,Acer saccharinum,L.,2616367,,,,2615354,1892-2,,N
4,Acer saccharum,sugar maple,2616387,56795-1,Species,Accepted,Sapindaceae,,Acer,,saccharum,,,,Marshall,,Arbust. Amer.,: 4,1785,,SE. Canada to C. & E. U.S.A. and NE. Mexico,tree,temperate,Acer saccharum,Marshall,2616387,,,,2615354,56795-1,,N


In [38]:
df_DB_WCVP_USDA_final.to_csv(project_data_path+'DB_WCVP_USDA_final.csv',index=False)